# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [ ]:
import os
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown, display, update_display

In [1]:
MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

In [ ]:
load_dotenv(override=True)

api_key = os.getenv("OPENROUTER_API_KEY")
if not api_key:
    raise ValueError("OPENROUTER_API_KEY not found in .env file")


# OpenAI client
openai_client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=api_key
)

# Ollama client (OpenAI-compatible endpoint)
ollama_client = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")

In [ ]:
system_prompt = """
You are an expert technical tutor.

Explain concepts clearly, concisely, and accurately.
Use simple language first, then add technical depth if helpful.
Provide practical examples when appropriate.
Break complex ideas into step-by-step explanations.
Avoid unnecessary jargon, but use correct terminology.
Assume the learner is smart but unfamiliar with the topic.
"""

In [ ]:
def ask_llm(question, client, model, system_prompt):
    """Stream a response from any OpenAI-compatible backend and display live."""
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": question},
    ]

    stream = client.chat.completions.create(
        model=model,
        messages=messages,
        stream=True,
    )

    response = ""
    display_handle = display(Markdown(""), display_id=True)

    for chunk in stream:
        delta = chunk.choices[0].delta.content
        if delta:
            response += delta
            update_display(
                Markdown(response),
                display_id=display_handle.display_id,
            )

    return response

In [ ]:
question = "What is an AI agent?"

In [ ]:
print("## GPT-4o-mini Response...\n")

user_input = input("Enter your question: ").strip()

user_question = user_input if user_input else question
try:
    gpt_response = ask_llm(user_question, openai_client, MODEL_GPT, system_prompt)
except Exception as e:
    print(f"GPT error: {e}")

In [ ]:
print("## Llama Response...\n")

user_input = input("Enter your question: ").strip()

user_question = user_input if user_input else question
try:
    llama_response = ask_llm(user_question, openai_client, MODEL_LLAMA, system_prompt)
except Exception as e:
    print(f"Llama error: {e}. Make sure Ollama is running and the model is available.")